In [1]:
import psycopg2
from psycopg2  import extras
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, Date

DATABASE = 'postgresql'
USER = 'root'
PASSWORD = 'root'
HOST = 'postgres' # shared-networkに所属しているpostgres containerを指定できる
PORT = '5432'
DB_NAME = 'sandbox'

CONNECT_STR = '{}://{}:{}@{}:{}/{}'.format(DATABASE, USER, PASSWORD, HOST, PORT, DB_NAME)
engine = create_engine(CONNECT_STR)

conn = engine.connect()
res = conn.execute("select datname, datdba, encoding, datcollate, datctype from pg_database;")

for row in res:
    print(row)

('postgres', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('sandbox', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template1', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template0', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')


In [2]:
# create table
tablename = 'x006_load_sample'

q1 = "SELECT * FROM information_schema.tables WHERE table_name = '{}';".format(tablename)
res = conn.execute(q1)

q2 = """
CREATE TABLE  {} (
    sample_date date NOT NULL,
    load_val integer NOT NULL
);
""".format(tablename)

if sum(1 for _ in res) == 0:
    print("create")
    conn.execute(q2)
    res = conn.execute(q1)
    sum(1 for _ in res)

In [3]:
# insert data
data = (
    ('2018-02-01', 1024),
    ('2018-02-02', 2366),
    ('2018-02-05', 2366),
    ('2018-02-07', 985),
    ('2018-02-08', 780),
    ('2018-02-12', 1000),
)

conn.execute("DELETE FROM {};".format(tablename))

metadata = MetaData()
tbl = Table(tablename, metadata,
            Column('sample_date', Date),
            Column('load_val', Integer),
           )

for i in data:
    ins = tbl.insert().values(sample_date=i[0], load_val=i[1])
    conn.execute(ins)

res = conn.execute("SELECT * FROM {};".format(tablename))
for i in res:
    print(i)

(datetime.date(2018, 2, 1), 1024)
(datetime.date(2018, 2, 2), 2366)
(datetime.date(2018, 2, 5), 2366)
(datetime.date(2018, 2, 7), 985)
(datetime.date(2018, 2, 8), 780)
(datetime.date(2018, 2, 12), 1000)


## ウィンドウ関数の3機能

- PARTITION BYによるレコード集合のカット
- ORDER BYによるレコードの順序づけ
- フレーム句によるカレントコードを中心としたサブセットの定義

In [4]:
q = f"""
select sample_date as cur_date,
    min(sample_date)
        over (order by sample_date asc
                rows between 1 preceding and 1 preceding) as latest_date
from {tablename};
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select sample_date as cur_date,
    min(sample_date)
        over (order by sample_date asc
                rows between 1 preceding and 1 preceding) as latest_date
from x006_load_sample;

(datetime.date(2018, 2, 1), None)
(datetime.date(2018, 2, 2), datetime.date(2018, 2, 1))
(datetime.date(2018, 2, 5), datetime.date(2018, 2, 2))
(datetime.date(2018, 2, 7), datetime.date(2018, 2, 5))
(datetime.date(2018, 2, 8), datetime.date(2018, 2, 7))
(datetime.date(2018, 2, 12), datetime.date(2018, 2, 8))


In [5]:
# postgres11以降でないと動かない

# 一日前に該当する場合だけ値が入る
q = f"""
select sample_date as cur_date,
    min(sample_date)
        over (order by sample_date asc
                range between interval '1' day preceding and interval '1' day preceding) as day1_before
from {tablename};
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select sample_date as cur_date,
    min(sample_date)
        over (order by sample_date asc
                range between interval '1' day preceding and interval '1' day preceding) as day1_before
from x006_load_sample;

(datetime.date(2018, 2, 1), None)
(datetime.date(2018, 2, 2), datetime.date(2018, 2, 1))
(datetime.date(2018, 2, 5), None)
(datetime.date(2018, 2, 7), None)
(datetime.date(2018, 2, 8), datetime.date(2018, 2, 7))
(datetime.date(2018, 2, 12), None)


In [6]:
# postgres11以降でないと動かない

# 1日前から1日後に当てはまるものの中で最小のものをとる、つまり1日前があればその日付が入る
q = f"""
select sample_date as cur_date,
    min(sample_date)
        over (order by sample_date asc
                range between interval '1' day preceding and interval '1' day following) as day1_before
from {tablename};
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select sample_date as cur_date,
    min(sample_date)
        over (order by sample_date asc
                range between interval '1' day preceding and interval '1' day following) as day1_before
from x006_load_sample;

(datetime.date(2018, 2, 1), datetime.date(2018, 2, 1))
(datetime.date(2018, 2, 2), datetime.date(2018, 2, 1))
(datetime.date(2018, 2, 5), datetime.date(2018, 2, 5))
(datetime.date(2018, 2, 7), datetime.date(2018, 2, 7))
(datetime.date(2018, 2, 8), datetime.date(2018, 2, 7))
(datetime.date(2018, 2, 12), datetime.date(2018, 2, 12))
